# Import SSBClient
Import the SSBClient class for querying the SSB API.

In [1]:
import sys
import pandas as pd
import numpy as np
sys.path.append('../common/ssb')
from client import SSBClient

# Query SSB API with patients.json
Load the query from patients.json and fetch the data using SSBClient.

In [18]:
import json
with open('../../config/queries/patients.json', 'r') as f:
    patients_query = json.load(f)

client = SSBClient()
df_patients = client.query('10261', patients_query)
df_patients.head()

,Region,Diagnose3,ContentsCode,Tid,value
0,Total,Acute respiratory infections,Number of in-patient stays (discharges),2012,11083
1,Total,Acute respiratory infections,Number of in-patient stays (discharges),2013,10589
2,Total,Acute respiratory infections,Number of in-patient stays (discharges),2014,9937
3,Total,Acute respiratory infections,Number of in-patient stays (discharges),2015,9033
4,Total,Acute respiratory infections,Number of in-patient stays (discharges),2016,9783


## Clean and Prepare Patient Data
Remove unnecessary columns and rename variables for consistency.

In [19]:
df_patients.drop(columns=['Diagnose3'], inplace=True)
df_patients.rename(columns={'Region': 'county', 'ContentsCode': 'variable', 'value': 'patient_count', 'Tid': "year"}, inplace=True)

In [ ]:
df_patients.to_csv('../../data/raw/patients.csv', index=False)

## Query Vehicle Data
Fetch vehicle registration data by fuel type and region from SSB API.

In [58]:
import json
with open('../../config/queries/vehicles.json', 'r') as f:
    patients_query = json.load(f)

client = SSBClient()
df_vehicles = client.query('11823', patients_query)
df_vehicles.head()

,Region,DrivstoffType,ContentsCode,Tid,value
0,Østfold,Petrol,Private cars,2016,0
1,Østfold,Petrol,Private cars,2017,0
2,Østfold,Petrol,Private cars,2018,0
3,Østfold,Petrol,Private cars,2019,0
4,Østfold,Petrol,Private cars,2020,0


## Clean and Prepare Vehicle Data
Rename columns for consistency with other datasets.

In [59]:
df_vehicles.rename(columns={'Region': 'county', 'DrivstoffType': 'engine_type', 'ContentsCode': 'vehicle_type', 'value': 'count', 'Tid': "year"}, inplace=True)


In [ ]:
df_vehicles.to_csv('../../data/raw/vehicles.csv', index=False)

## Query Population Data
Fetch population statistics by region from SSB API.

In [4]:
import json
with open('../../config/queries/population.json', 'r') as f:
    patients_query = json.load(f)

client = SSBClient()
df_population = client.query('06913', patients_query)
df_population.head()

,Region,ContentsCode,Tid,value
0,Østfold,Population 1 January,1951,0
1,Østfold,Population 1 January,1952,0
2,Østfold,Population 1 January,1953,0
3,Østfold,Population 1 January,1954,0
4,Østfold,Population 1 January,1955,0


## Clean and Prepare Population Data
Remove unnecessary columns and rename for consistency.

In [ ]:
df_population.drop(columns=['ContentsCode'], inplace=True)
df_population.rename(columns={'Region': 'county', 'value': 'population', 'Tid': "year"}, inplace=True)

In [8]:
df_population.to_csv('../../data/raw/population.csv', index=False)

---

# Data Transformation and Harmonization

## County Mapping
Create a mapping to unify county names across different time periods, accounting for administrative reforms and mergers.

In [9]:
# Map counties to unified county names (merging those that changed due to administrative reforms)
county_unify_map = {
    'Total': 'total',
    'Østfold': 'viken',
    'Østfold (-2019)': 'viken',
    'Akershus': 'viken',
    'Akershus (-2019)': 'viken',
    'Viken (2020-2023)': 'viken',
    'Oslo': 'oslo',
    'Innlandet': 'innlandet',
    'Hedmark (-2019)': 'innlandet',
    'Oppland (-2019)': 'innlandet',
    'Buskerud': 'viken',
    'Buskerud (-2019)': 'viken',
    'Vestfold': 'vestfold_telemark',
    'Vestfold (-2019)': 'vestfold_telemark',
    'Telemark': 'vestfold_telemark',
    'Telemark (-2019)': 'vestfold_telemark',
    'Vestfold og Telemark (2020-2023)': 'vestfold_telemark',
    'Agder': 'agder',
    'Aust-Agder (-2019)': 'agder',
    'Vest-Agder (-2019)': 'agder',
    'Rogaland': 'rogaland',
    'Vestland': 'vestland',
    'Hordaland (-2019)': 'vestland',
    'Sogn og Fjordane (-2019)': 'vestland',
    'Møre og Romsdal': 'more_romsdal',
    'Trøndelag - Trööndelage': 'trondelag',
    'Sør-Trøndelag (-2017)': 'trondelag',
    'Nord-Trøndelag (-2017)': 'trondelag',
    'Nordland - Nordlánnda': 'nordland',
    'Troms - Romsa - Tromssa': 'troms_finnmark',
    'Troms - Romsa (-2019)': 'troms_finnmark',
    'Finnmark - Finnmárku - Finmarkku': 'troms_finnmark',
    'Finnmark - Finnmárku (-2019)': 'troms_finnmark',
    'Troms og Finnmark - Romsa ja Finnmárku (2020-2023)': 'troms_finnmark',
    'Unknown': 'unknown'
}


## Transform Patient Data
Apply county mapping, rename variables, pivot the data structure, and aggregate by unified counties.

In [33]:
# Load pre-saved patient data
df_patients = pd.read_csv('../../data/raw/patients.csv')
# Assign unique region IDs
df_patients['county_unified'] = df_patients['county'].map(county_unify_map)
# Map variable names to more convenient ones
df_patients["variable"] = df_patients.variable.map({'Number of in-patient stays (discharges)': 'inpatient_stays', 'Number of out-patient consultations': 'outpatient_consultations'})
# pivot longer at variable
df_patients = df_patients.pivot_table(index=['year', 'county', 'county_unified'], columns='variable', values='patient_count', fill_value=0).reset_index()
# aggregate to yearly counts by reduced county
df_patients = df_patients.groupby(['year', 'county_unified']).agg({'inpatient_stays': "sum", 'outpatient_consultations': "sum"}).reset_index()
# write to csv
df_patients.to_csv('../../data/patients.csv', index=False)

## Engine Type Mapping
Classify engine types into broader categories: ICEV (Internal Combustion Engine Vehicles), BEV (Battery Electric Vehicles), and HEV (Hybrid Electric Vehicles).

In [12]:
engine_type_map = {
    'Diesel': 'ICEV',
    'Petrol': 'ICEV',
    'Gas': 'ICEV',
    'Paraffin': 'ICEV',
    'Other fuel': 'ICEV',
    'Electricity': 'BEV',
    'Hydrogen': 'BEV',
    'Diesel hybrid, chargeable': 'HEV',
    'Diesel hybrid, non-chargeable': 'HEV',
    'Petrol hybrid, chargeable': 'HEV',
    'Petrol hybrid, non-chargeable': 'HEV'
}

## Transform Vehicle Data
Apply county and engine type mappings, aggregate data, and restructure for analysis.

In [34]:
# Load pre-saved vehicle data
df_vehicles = pd.read_csv('../../data/raw/vehicles.csv')
# Assign unique region IDs
df_vehicles['county_unified'] = df_vehicles['county'].map(county_unify_map)
# aggregate to yearly counts by reduced county
df_vehicles = df_vehicles.groupby(['year', 'county_unified', 'engine_type']).agg({'count': "sum"}).reset_index()
# aggregate engine types
df_vehicles['engine_type_unified'] = df_vehicles['engine_type'].map(engine_type_map)
df_vehicles = df_vehicles.groupby(['year', 'county_unified', 'engine_type_unified']).agg({'count': "sum"}).reset_index()
# pivot wider on engine type
df_vehicles = df_vehicles.pivot_table(index=['year', 'county_unified'], columns='engine_type_unified', values='count', fill_value=0).reset_index()
# write to csv
df_vehicles.to_csv('../../data/vehicles.csv', index=False)

## Transform Population Data
Apply county mapping and aggregate population data by unified counties.

In [35]:
# Load pre-saved population data
df_population = pd.read_csv('../../data/raw/population.csv')
# Assign unique region IDs
df_population['county_unified'] = df_population['county'].map(county_unify_map)
# aggregate to yearly counts by reduced county
df_population = df_population.groupby(['year', 'county_unified']).agg({'population': "sum"}).reset_index()
# write to csv
df_population.to_csv('../../data/population.csv', index=False)

---

# Data Integration and Final Dataset Creation

## Merge Datasets
Combine patient, population, and vehicle datasets on year and unified county identifiers.

In [54]:
# merge datasets
df_merged = df_patients.merge(df_population, on=['year', 'county_unified'], how='left')
df_merged = df_merged.merge(df_vehicles, on=['year', 'county_unified'], how='left')

## Calculate Per-Capita Metrics
Compute health outcomes and vehicle ownership rates per 1,000 inhabitants, as well as vehicle share metrics.

In [55]:
# Compute per-capita values
df_merged['inpatient_stays_per_1000'] = df_merged['inpatient_stays'] / df_merged['population'] * 1000
df_merged['outpatient_consultations_per_1000'] = df_merged['outpatient_consultations'] / df_merged['population'] * 1000
df_merged['BEV_per_1000'] = df_merged['BEV'] / df_merged['population'] * 1000
df_merged['HEV_per_1000'] = df_merged['HEV'] / df_merged['population'] * 1000
df_merged['ICEV_per_1000'] = df_merged['ICEV'] / df_merged['population'] * 1000
df_merged['CEV_per_1000'] = (df_merged['HEV'] + df_merged['BEV']) / df_merged['population'] * 1000
df_merged['BEV_share'] = df_merged['BEV'] / (df_merged['BEV'] + df_merged['HEV'] + df_merged['ICEV'])

## Save Final Dataset
Export the merged and processed dataset for analysis.

In [56]:
# Write final merged dataset to csv
df_merged.to_csv('../../data/merged_data.csv', index=False)

---
## Regression Analysis

### Import Regression Libraries
Load necessary libraries for panel data regression analysis.

In [62]:
import linearmodels.panel as plm
import statsmodels.api as sm

### Load Analysis Dataset
Import the final merged dataset for regression analysis.

In [57]:
df_merged = pd.read_csv('../../data/merged_data.csv')

### Inpatient Stays Regression
Panel regression analysis examining the relationship between BEV adoption and inpatient hospital stays.

In [68]:
# Run regression analysis with robust standard errors
mod = plm.PanelOLS.from_formula('inpatient_stays_per_1000 ~ BEV_per_1000 + EntityEffects + TimeEffects', data=df_merged.set_index(['county_unified', 'year']))
res = mod.fit(cov_type='robust')
res.summary

/Users/felixschulz/miniforge3/envs/evs-and-health/lib/python3.13/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,inpatient_stays_per_1000,R-squared:,0.0754
Estimator:,PanelOLS,R-squared (Between):,-0.2906
No. Observations:,99,R-squared (Within):,0.0270
Date:,"Mon, Sep 29 2025",R-squared (Overall):,-0.2801
Time:,14:47:40,Log-likelihood,54.122
Cov. Estimator:,Robust,,
,,F-statistic:,6.4384
Entities:,11,P-value,0.0131
Avg Obs:,9.0000,Distribution:,"F(1,79)"
Min Obs:,9.0000,,
Max Obs:,9.0000,F-statistic (robust):,6.0419


### Outpatient Consultations Regression
Panel regression analysis examining the relationship between BEV adoption and outpatient consultations.

In [69]:
# Run regression analysis with robust standard errors
mod = plm.PanelOLS.from_formula('outpatient_consultations_per_1000 ~ BEV_per_1000 + EntityEffects + TimeEffects', data=df_merged.set_index(['county_unified', 'year']))
res = mod.fit(cov_type='robust')
res.summary

/Users/felixschulz/miniforge3/envs/evs-and-health/lib/python3.13/site-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


Dep. Variable:,outpatient_consultations_per_1000,R-squared:,8.262e-06
Estimator:,PanelOLS,R-squared (Between):,0.0074
No. Observations:,99,R-squared (Within):,-0.0018
Date:,"Mon, Sep 29 2025",R-squared (Overall):,0.0069
Time:,14:48:33,Log-likelihood,-51.143
Cov. Estimator:,Robust,,
,,F-statistic:,0.0007
Entities:,11,P-value,0.9797
Avg Obs:,9.0000,Distribution:,"F(1,79)"
Min Obs:,9.0000,,
Max Obs:,9.0000,F-statistic (robust):,0.0004


### Alternative Model: Clean Electric Vehicles (CEV)
OLS regression examining the combined effect of BEV and HEV (Clean Electric Vehicles) on outpatient consultations.

In [60]:
smf.ols('outpatient_consultations_per_1000 ~ CEV_per_1000 + C(year) + C(county_unified)', data=df_merged).fit(cov_type='HC3').summary()

/Users/felixschulz/miniforge3/envs/evs-and-health/lib/python3.13/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 25, but rank is 20
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                                    OLS Regression Results                                   
=============================================================================================
Dep. Variable:     outpatient_consultations_per_1000   R-squared:                       0.585
Model:                                           OLS   Adj. R-squared:                  0.485
Method:                                Least Squares   F-statistic:                     289.6
Date:                               Mon, 29 Sep 2025   Prob (F-statistic):           2.05e-65
Time:                                       14:36:40   Log-Likelihood:                -49.496
No. Observations:                                 99   AIC:                             139.0
Df Residuals:                                     79   BIC:                             190.9
Df Model:                                         19                                         
Covariance Type:                                 HC3                                         
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.9304      0.549      1.694      0.090      -0.146       2.007
C(year)[T.2013]                        -1.372e-14   1.12e-14     -1.223      0.221   -3.57e-14    8.26e-15
C(year)[T.2014]                          3.16e-16    4.2e-15      0.075      0.940   -7.93e-15    8.56e-15
C(year)[T.2015]                         5.529e-16    4.1e-15      0.135      0.893   -7.49e-15    8.59e-15
C(year)[T.2016]                            0.6920      0.384      1.803      0.071      -0.060       1.444
C(year)[T.2017]                            0.6822      0.295      2.312      0.021       0.104       1.261
C(year)[T.2018]                            0.5078      0.206      2.469      0.014       0.105       0.911
C(year)[T.2019]                            0.4082      0.142      2.874      0.004       0.130       0.687
C(year)[T.2020]                           -0.3042      0.253     -1.201      0.230      -0.801       0.192
C(year)[T.2021]                           -0.0708      0.170     -0.417      0.676      -0.403       0.262
C(year)[T.2022]                           -0.0832      0.395     -0.210      0.833      -0.858       0.692
C(year)[T.2023]                           -0.3293      0.508     -0.648      0.517      -1.325       0.666
C(year)[T.2024]                           -0.5725      0.646     -0.886      0.376      -1.839       0.694
C(county_unified)[T.innlandet]             0.0145      0.194      0.075      0.940      -0.365       0.394
C(county_unified)[T.more_romsdal]          0.1392      0.141      0.984      0.325      -0.138       0.416
C(county_unified)[T.nordland]              0.1961      0.195      1.008      0.314      -0.185       0.577
C(county_unified)[T.oslo]                  0.4983      0.348      1.431      0.152      -0.184       1.181
C(county_unified)[T.rogaland]             -0.0682      0.140     -0.487      0.626      -0.343       0.206
C(county_unified)[T.total]              8.444e-18   6.14e-17      0.138      0.891   -1.12e-16    1.29e-16
C(county_unified)[T.troms_finnmark]       -0.1488      0.284     -0.524      0.600      -0.706       0.408
C(county_unified)[T.trondelag]             0.7788      0.129      6.057      0.000       0.527       1.031
C(county_unified)[T.unknown]           -4.255e-17   3.33e-17     -1.277      0.202   -1.08e-16    2.28e-17
C(county_unified)[T.vestfold_telemark]     0.0124      0.121      0.103      0.918      -0.224       0.249
C(county_unified)[T.vestland]              0.6263      0.223      2.812      0.005       0.190       1.063
C(county_unified)[T.viken]   